In [1]:
%%configure -f 
{
 "jars": [],
 "name":"session-vlad",
 "numExecutors": 2,
 "executorMemory": "1g",
 "driverMemory": "3g",    
 "conf": {
    "spark.driver.maxResultSize": "2g",
    "livy.server.yarn.app-lookup-timeout": "300s"
 }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
64,application_1533135836622_1290,spark,idle,Link,Link,
65,application_1533135836622_1291,spark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
67,application_1533135836622_1297,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3e74de8d

In [3]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.graphx.util.GraphGenerators

import org.apache.spark.graphx.util.GraphGenerators

In [4]:
case class Geohash(gh: String, dist: Option[Double] = None)

defined class Geohash

In [73]:
val eventsRdd: RDD[(VertexId, Geohash)] =
  sc.parallelize(
    Array((1L, Geohash("s")), (2L, Geohash("t")), 
          (3L, Geohash("y")), (4L, Geohash("x")), 
          (5L, Geohash("z")))
  )
val relationshipsRdd: RDD[Edge[Double]] = 
    sc.parallelize(
        Array(Edge(1L, 2L, 6.0), 
              Edge(1L, 3L, 7.0),
              Edge(2L, 3L, 8.0), 
              Edge(2L, 4L, 5.0), 
              Edge(4L, 2L, -2.0),
              Edge(2L, 5L, -4.0),
              Edge(5L, 4L, 7.0),
              Edge(5L, 1L, 2.0),
              Edge(3L, 5L, 9.0),
              Edge(3L, 4L, -3.0)                                        
             )
    )
val graph: Graph[Geohash, Double] = Graph(eventsRdd, relationshipsRdd)

graph: org.apache.spark.graphx.Graph[Geohash,Double] = org.apache.spark.graphx.impl.GraphImpl@14a9045e

In [74]:
val facts: RDD[String] =
  graph.triplets.map(triplet =>
   s"${triplet.srcAttr.gh}  ${triplet.dstAttr.gh} ${triplet.attr}")
facts.collect.foreach(println(_))

s  t 6.0
s  y 7.0
t  y 8.0
t  x 5.0
x  t -2.0
t  z -4.0
z  s 2.0
z  x 7.0
y  x -3.0
y  z 9.0

$ vprog: (VertexId, VD, A) => VD$

$ sendMsg: EdgeTriplet[VD, ED] => Iterator[(VertexId, A)]$

$  mergeMsg: (A, A) => A $

In [85]:
val sendMsg: (EdgeTriplet[Geohash, Double] => Iterator[(VertexId, Double)]) = { t =>     
     (t.srcAttr.dist, t.dstAttr.dist) match {
         case (Some(srcDist), None) => Iterator((t.dstId, srcDist + t.attr))
         case (Some(srcDist), Some(targetDist)) if  targetDist > srcDist + t.attr => Iterator((t.dstId, srcDist + t.attr))
         case _ => Iterator.empty                 
     }       
}

val mergeMsg: ((Double, Double) => Double) = (x, y) => math.min(x, y)

val vprog: ((VertexId, Geohash, Double) => Geohash) = (vid, gh, newDist) => {
    gh.dist match { 
        case None => gh.copy(dist = Some(newDist))
        case Some(d) if(d > newDist) => gh.copy(dist=Some(newDist))
        case _ => gh
    }
}

vprog: (org.apache.spark.graphx.VertexId, Geohash, Double) => Geohash = <function3>

In [86]:
val sourceId = 1L
val initialGraph: Graph[Geohash, Double] = graph.mapVertices((id, x) => if (id == sourceId) Geohash(x.gh, Some(0.0)) else x)
val ssp = initialGraph.pregel(initialMsg=Double.PositiveInfinity,
                              maxIterations = 100000                        
                             )(vprog, sendMsg, mergeMsg )

ssp.vertices.map{
    case (id, v) => s"${v.gh} shortest dist: ${v.dist.get} "}.collect.foreach(println(_))

s shortest dist: 0.0 
t shortest dist: 2.0 
y shortest dist: 7.0 
x shortest dist: 4.0 
z shortest dist: -2.0